In [1]:
# !rm -r /home/karem/DatasetKaggleCovid/Dataset/Original/.ipynb_checkpoints
# !rm -fr /home/karem/.local/share/Trash/files/*

In [1]:
import pandas as pd
import os
import numpy as np

from pathlib import Path

### Funções

In [2]:
# se o diretório não existir, ele é criado
def createfolder(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
        
# se houver a pasta, ela é removida
import shutil
def remove_folders(path):
    if os.path.exists(path):
        shutil.rmtree(path)

In [3]:
###############################################
### Separa os dados em treino e teste
###############################################

# 90% dos dados são utilizados para treino, 0% para validação e 10% para teste
def train_validate_test_split(df, train_percent=.9, validate_percent=.0, seed=42): # sem validação
#def train_validate_test_split(df, train_percent=.85, validate_percent=.05, seed=42): # com validação
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [4]:
def remove_negative_cases(df):
    
    indexNames = df[(df['class_name'] == 'negative')].index
    df.drop(indexNames , inplace=True)

    df = df.reset_index(drop=True) # reinicia o index

    df = df.fillna(0) # substitui NaN por zero

    # print(len(df))
    # df.head(5)
    return df

In [5]:
def convert_toRGB(img, dest, name):
    img = img.convert(mode='L')
    array = np.asarray(img) ## -> converte para array  
    
    # Convert to 8 bits
    array = array - np.min(array)
    array = array / np.max(array)
    array = (array * (2**8-1)).astype(np.uint8)               
    array = cv2.cvtColor(array, cv2.COLOR_GRAY2RGB)    
    
    cv2.imwrite(dest + name + '.jpg', array)

In [6]:
from PIL import Image
def save_image(df, dest):
    for i in range(len(df)):
        name = df['id'][i].replace('_image', '.png')
        im = Image.open(path_dataset + name)        
        im.save(dest + name)

In [7]:
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [8]:
def convYolo(xmin, ymin, w, w_img, h, h_img):
    xcenter = (xmin + w/2) / w_img
    ycenter = (ymin + h/2) / h_img
    w = w / w_img
    h = h / h_img
    return (xcenter, ycenter, w, h)

In [9]:
import re
def createlabels(split, df):
    if size_image == 'ori':
        for i in range(len(df)): #2):
            M = []
            #f = open(split + (df['id'][i]).replace('_image', '.txt'), "w+") # cria o arquivo txt no diretório indicado e com o nome da imagem

            # percorre a quantidade de rótulos/labels
            for j in range(len(df['xmin_ori'][i])):
                l = []

                # verifica a se tem label. Se tiver recebe a classe 0 (= covid)
                # senão recebe 1 (negativo)
                if df['class_name'][i] == 'negative':
                    classes = 1
                else:
                    classes = 0

                # adiciona a classe e os rótulos na lista
                l.append(classes)
                bb = convYolo(df['xmin_ori'][i][j], df['ymin_ori'][i][j], df['width_roi_ori'][i][j], \
                              df['width_ori'][i], df['height_roi_ori'][i][j], df['height_ori'][i]) 
                l.append(bb[0])
                l.append(bb[1])
                l.append(bb[2])
                l.append(bb[3])

                M.append(l)
                M[j][0] = np.round(M[j][0],0)
            np.savetxt(split + (df['id'][i]).replace('_image', '.txt'), M, fmt=['%d', '%s', '%s', '%s', '%s']) #fmt="%s")
    else:
        for i in range(len(df)): #2):
            M = []
            
            # percorre a quantidade de rótulos/labels
            for j in range(len(df['xmin'][i])):
                l = []

                # verifica a se tem label. Se tiver recebe a classe 0 (= covid)
                # senão recebe 1 (negativo)
                if df['class_name'][i] == 'negative':
                    classes = 1
                else:
                    classes = 0

                # adiciona a classe e os rótulos na lista
                l.append(classes)

                bb = convYolo(df['xmin'][i][j], df['ymin'][i][j], df['width_roi'][i][j], df['width'][i], \
                              df['height_roi'][i][j], df['height'][i]) 
                l.append(bb[0])
                l.append(bb[1])
                l.append(bb[2])
                l.append(bb[3])

                M.append(l)
                M[j][0] = np.round(M[j][0],0)

            np.savetxt(split + (df['id'][i]).replace('_image', '.txt'), M, fmt=['%d', '%s', '%s', '%s', '%s']) #fmt="%s")


### Início

In [10]:
## Captura o diretório atual e armazena em uma variável
diretorio = os.getcwd() + '/'
print(diretorio)


#############################
# Escolhe se vai trabalhar a imagem no tamanho original ou reduzido
###############
# size_image = 'ori' # com os dados do tamanho original
size_image = 'resize' # com os dados do tamanho reduzido (512x512) ou (416x416)

#### Se for reduzido, definir o tamanho
size = 640 # 416, 512 ou 640 --> yolo tem por padrão o 640


#############################
# Escolhe o tipo de treinamento - se terá ou não casos negativos
#############################
# train = 1 => 2class_TrainCovidNormal
# train = 2 => 2class_TrainCovid
# train = 3 => 1class_trainCovid
tipo_train = 3

/home/karem/Artigo_Periodico/Detector/YoloV5/1_Divide_Dados_Train_Test/


In [11]:
###############################################################################################
# Carrega o csv que contém apenas os casos negativos ou com bounding boxes delimitados
# (há casos positivos que o radiologista não delimitou os BB)
###############################################################################################

dircsv = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/'
df = pd.read_csv(dircsv + 'CSV_CasosSemBBRemovidos.csv')
print(len(df))
#df.head(5)

9589


In [12]:
# Remove os casos atypical = outras doenças (mantém covid - typical e indeterminate; e negativos)
df2 = df.copy()
indexNames = df2[(df2['class_name'] == 'atypical')].index
df2.drop(indexNames , inplace=True)

df2 = df2.reset_index(drop=True) # reinicia o index

df2 = df2.fillna(0) # substitui NaN por zero

print(len(df2))
#df2.head(5)

9052


In [13]:
######### df3 = com os dados do tamanho original
if size_image == 'ori':
    ### yolo => [xmid, ymid, w, h] (normalized)
    df3 = df2[['id', 'class_name', 'xmin_ori', 'ymin_ori', 'xmax_ori', 'ymax_ori', 'width_roi_ori', \
           'height_roi_ori', 'width_ori', 'height_ori']].copy()
    #df3 = df3.groupby('id', as_index=False).agg(lambda x: x.tolist())

    # concatena linhas com o mesmo 'id'
    df3 = df3.groupby(['id', 'class_name', 'width_ori', 'height_ori'], as_index=False)['xmin_ori', \
                                        'ymin_ori', 'xmax_ori', 'ymax_ori', 'width_roi_ori', 'height_roi_ori'].agg(list)
    df3.columns = ['id', 'class_name', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax', 'width_roi', 'height_roi']
    
######### df3 = com os dados do tamanho reduzido (640x640)
else:
    df3 = df2[['id', 'class_name', 'xmin', 'ymin', 'xmax', 'ymax', 'width_roi', \
               'height_roi']].copy()

    df3['width'] = size
    df3['height'] = size

    df3 = df3.groupby(['id', 'class_name', 'width', 'height'], as_index=False)['xmin', \
                                             'ymin', 'xmax', 'ymax', 'width_roi', 'height_roi'].agg(list)

print(len(df3))
df3.head(5)

/tmp/ipykernel_11309/1655492580.py:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df3.groupby(['id', 'class_name', 'width', 'height'], as_index=False)['xmin', \


5639


,id,class_name,width,height,xmin,ymin,xmax,ymax,width_roi,height_roi
0,000a312787f2_image,typical,640,640,"[119.0, 338.0]","[107.0, 108.0]","[273.0, 502.0]","[459.0, 432.0]","[154.0, 165.0]","[352.0, 323.0]"
1,000c3a3f293f_image,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
2,0012ff7358bc_image,typical,640,640,"[142.0, 375.0]","[50.0, 101.0]","[324.0, 505.0]","[301.0, 404.0]","[182.0, 129.0]","[252.0, 303.0]"
3,001bd15d1891_image,typical,640,640,"[117.0, 484.0]","[240.0, 228.0]","[251.0, 609.0]","[493.0, 484.0]","[134.0, 124.0]","[253.0, 256.0]"
4,0022227f5adf_image,indeterminate,640,640,[390.0],[128.0],[469.0],[229.0],[79.0],[101.0]


In [14]:
print('Typical cases: ', df3.class_name.str.contains('typical').sum())
print('Indeterminate cases: ', df3.class_name.str.contains('indeterminate').sum())
print('Negative cases: ', df3.class_name.str.contains('negative').sum())
print('Atypical cases: ', df3.class_name.str.contains('atypical').sum())

Typical cases:  2854
Indeterminate cases:  1049
Negative cases:  1736
Atypical cases:  0


### Divide os dados de treino e teste

In [15]:
# Divide o conjunto em treino e teste
train, validate, test = train_validate_test_split(df3) # mantém os casos negativos
train = train.reset_index(drop=True) # reinicia o index
validate = validate.reset_index(drop=True) # reinicia o index
test = test.reset_index(drop=True) # reinicia o index
print(len(train), len(validate), len(test))
train.head(5)

5075 0 564


,id,class_name,width,height,xmin,ymin,xmax,ymax,width_roi,height_roi
0,611964e17dd1_image,typical,640,640,"[126.0, 398.0]","[351.0, 365.0]","[270.0, 519.0]","[498.0, 499.0]","[144.0, 122.0]","[147.0, 134.0]"
1,499ba19916d1_image,typical,640,640,"[70.0, 379.0]","[278.0, 275.0]","[261.0, 559.0]","[623.0, 604.0]","[191.0, 180.0]","[345.0, 329.0]"
2,9f126fed5074_image,indeterminate,640,640,[414.0],[343.0],[616.0],[490.0],[202.0],[147.0]
3,eda0821322f7_image,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
4,041c956d9bd7_image,typical,640,640,"[501.0, 56.0]","[194.0, 156.0]","[639.0, 184.0]","[502.0, 442.0]","[138.0, 128.0]","[309.0, 285.0]"


### Gerar dados de validação

In [16]:
# Cria uma pasta contendo as imagens -> train, val e test

if size_image == 'ori':
    path_dataset = '/home/karem/DatasetKaggleCovid/Dataset/dataOriginal_PreProcessing/train/'
    
else:
    path_dataset = '/home/karem/Artigo_Periodico/1_Dataset/2_PreProc_size640/'

In [17]:
df_trainval = train.copy()
df_test = test.copy()

print('Size train/val: ', len(df_trainval))
print('---------------------------')

# Calcula o tamanho do dataframe de validação - 5%
size = round(len(df_trainval) * 5 / 100) # o tamanho do df da validação será de 5% do tatal das imagens de treino
df_val = df_trainval.iloc[:size, :]
df_val = df_val.reset_index(drop=True) # reinicia o index

df_train = df_trainval.iloc[size:, :]
df_train = df_train.reset_index(drop=True) # reinicia o index

print('Size train: ', len(df_train))
print('Size val: ', len(df_val))
print('Size test: ', len(df_test))

Size train/val:  5075
---------------------------
Size train:  4821
Size val:  254
Size test:  564


In [18]:
#################################################################
# Remove os casos negativos apenas do conjunto de treino/val 

if tipo_train == 2:     
    df_trainval = remove_negative_cases(df_trainval)
    df_train = remove_negative_cases(df_train)
    df_val = remove_negative_cases(df_val)    
elif tipo_train == 3:
    df_trainval = remove_negative_cases(df_trainval)
    df_train = remove_negative_cases(df_train)
    df_val = remove_negative_cases(df_val)
    df_test = remove_negative_cases(df_test)   

print('Size train/val: ', len(df_trainval))
print('Size train: ', len(df_train))
print('Size val: ', len(df_val))
print('Size test: ', len(df_test))

Size train/val:  3514
Size train:  3331
Size val:  183
Size test:  389


In [19]:
if tipo_train == 1:
    folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/2class_trainCovidNormal/'
elif tipo_train == 2:
    folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/2class_trainCovid/'
elif tipo_train == 3:
    folder = '/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/1class_trainCovid/'

remove_folders(folder)
createfolder(folder)

# Salva o csv de treino e validação
df_trainval.to_csv(folder + 'trainval.csv',
           index = False, sep=',')

# Salva o csv de treino (exclui os dados da validação do csv)
df_train.to_csv(folder + 'train.csv', 
           index = False, sep=',')

# Salva o csv de validação
df_val.to_csv(folder + 'val.csv', 
           index = False, sep=',')

# Salva o csv de teste
df_test.to_csv(folder + 'test.csv', 
           index = False, sep=',')

In [21]:
print('Train/Val -> Typical cases: ', df_trainval.class_name.str.contains('typical').sum())
print('Train/Val -> Indeterminate cases: ', df_trainval.class_name.str.contains('indeterminate').sum())
print('Train/Val -> Negative cases: ', df_trainval.class_name.str.contains('negative').sum())
print('Train/Val -> Atypical cases: ', df_trainval.class_name.str.contains('atypical').sum())

print('_________________________________________')
print('Test -> Typical cases: ', df_test.class_name.str.contains('typical').sum())
print('Test -> Indeterminate cases: ', df_test.class_name.str.contains('indeterminate').sum())
print('Test -> Negative cases: ', df_test.class_name.str.contains('negative').sum())
print('Test -> Atypical cases: ', df_test.class_name.str.contains('atypical').sum())

Train/Val -> Typical cases:  2568
Train/Val -> Indeterminate cases:  946
Train/Val -> Negative cases:  0
Train/Val -> Atypical cases:  0
_________________________________________
Test -> Typical cases:  286
Test -> Indeterminate cases:  103
Test -> Negative cases:  0
Test -> Atypical cases:  0


### Imagens

In [22]:
dest_train = diretorio + 'images_' + str(tipo_train) + '/train/'
dest_val = diretorio + 'images_' + str(tipo_train) + '/valid/'
dest_test = diretorio + 'images_' + str(tipo_train) + '/test/'

remove_folders(dest_train)
remove_folders(dest_val)
remove_folders(dest_test)

createfolder(dest_train)
createfolder(dest_val)
createfolder(dest_test)

############ Salva a imagem em formato JPG

print('início train')
save_image(df_train, dest_train)

print('início val')
save_image(df_val, dest_val)

print('início test')
save_image(df_test, dest_test)

início train
início val
início test


### Gerar txt Yolo - Labels

In [23]:
folder = diretorio

path_train = folder + 'labels_' + str(tipo_train) + '/train/'
path_val = folder + 'labels_' + str(tipo_train) + '/val/'
path_test = folder + 'labels_' + str(tipo_train) + '/test/'

createfolder(path_train)
createfolder(path_val)
createfolder(path_test)

createlabels(path_train, df_train)
createlabels(path_val, df_val)
createlabels(path_test, test)